In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from tqdm import tqdm

In [2]:
# Copy the dataset.zip and unzip it
!cp "/content/drive/MyDrive/Microchip/dataset.zip" "/content"
!unzip dataset.zip

Archive:  dataset.zip
  inflating: dataset/pcb_1.jpg       
  inflating: dataset/pcb_2.jpg       
  inflating: dataset/pcb_3.jpg       
  inflating: dataset/pcb_4.jpg       
  inflating: dataset/pcb_5.jpg       
  inflating: dataset/pcb_6.jpg       
   creating: dataset/typeA/
   creating: dataset/typeA/test/
  inflating: dataset/typeA/test/annotations.csv  
   creating: dataset/typeA/train/
  inflating: dataset/typeA/train/annotations.csv  
   creating: dataset/typeB/
   creating: dataset/typeB/test/
  inflating: dataset/typeB/test/annotations.csv  
   creating: dataset/typeB/train/
  inflating: dataset/typeB/train/annotations.csv  
   creating: dataset/typeC/
   creating: dataset/typeC/test/
  inflating: dataset/typeC/test/annotations.csv  
   creating: dataset/typeC/train/
  inflating: dataset/typeC/train/annotations.csv  
   creating: dataset/typeD/
   creating: dataset/typeD/test/
  inflating: dataset/typeD/test/annotations.csv  
   creating: dataset/typeD/train/
  inflating: data

In [3]:
def load_dataset(filename):
    # Read the .csv
    df = pd.read_csv(filename, delimiter=';')
    # Drop columns that we don't need
    df_cleaned = df.drop(df.columns[[0, 1, 2, 3]],axis = 1)
    # Set Dataframe's dtype to float32
    df_cleaned = df_cleaned.astype(np.float32)

    return df_cleaned

In [4]:
class MicrochipDataset(Dataset):
    def __init__(self, pressure_ts, seq_length=8, mean=0, std=1.0):
        self.data = torch.tensor(pressure_ts, dtype=torch.float)        
        self.data = (self.data - mean) / std
        self.seq_length = seq_length
        
    def __len__(self):
        return len(self.data) - self.seq_length - 1
    
    def __getitem__(self, i):
        return self.data[i:(i + self.seq_length)], self.data[i + self.seq_length]

In [5]:
class RegressionLSTM(nn.Module):
    def __init__(self, device, seq_length, hidden_units):
        super().__init__()
        self.seq_length = seq_length
        self.hidden_units = hidden_units
        self.num_layers = 1
        self.lstm = nn.LSTM(
            input_size=1,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )
        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)
        self.device = device

    def forward(self, x):
        x = x.unsqueeze(2)
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units, device=self.device).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units, device=self.device).requires_grad_()
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.
        
        return out

In [6]:
types = ['typeA', 'typeB', 'typeC', 'typeD', 'typeE']

In [11]:
for type in types:
    # Define train csv filename
    train_filename = f"/content/dataset/{type}/train/annotations.csv"
    test_filename = f"/content/dataset/{type}/test/annotations.csv"

    print(f"\nPerforming time series analysis for {type}\n")

    train_df = load_dataset(train_filename)
    test_df = load_dataset(test_filename)

    feature_types = ['50um', '20um']
    # seq_lengths = list(range(1,9))
    seq_lengths = [1]

    for feature in feature_types:
        with open(f"lstm_logs_{feature}_{type}.txt",'w',encoding = 'utf-8') as f:
            seq_losses = []
            for i, seq_len in enumerate(seq_lengths, start=1):

                # Get the feature of interest
                train_time_series = train_df[feature]
                test_time_series = test_df[feature]
                train_ts = train_time_series.to_numpy()
                test_ts = test_time_series.to_numpy()

                # Define the Sequence length
                seq_length = seq_len

                # Get mean and std od training data
                mean = np.mean(train_ts)
                std = np.std(train_ts)

                # Get the device
                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

                # Define the hyperparameters
                num_epochs = 50
                batch_size = 512
                ln_rate = 0.01
                num_hidden_units = 32

                # Create train and test datasets
                train_dataset = MicrochipDataset(train_ts, seq_length=seq_length, mean=mean, std=std)
                test_dataset = MicrochipDataset(test_ts, seq_length=seq_length, mean=mean, std=std)

                # Create dataloaders
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
                test_loader_in_sample = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
                
                # Initialize the model ,place it to device and set it to train mode
                lstm_model = RegressionLSTM(device=device, seq_length=seq_length, hidden_units=num_hidden_units)
                lstm_model.to(device)
                lstm_model.train()

                # Define loss function and optimizer 
                loss_function = nn.MSELoss()
                optimizer = torch.optim.Adam(lstm_model.parameters(), lr=ln_rate)

                print(f"\nTraining for {feature} with sequence length = {seq_length}\n")

                # Training Loop
                for epoch in tqdm(range(num_epochs), total=num_epochs):
                    epoch_loss = 0
                    for X, y in train_loader:
                        
                        # Place the training sample to device
                        X_dev = X.to(device)
                        y_dev = y.to(device)

                        # Get the prediction and calculate the loss
                        prediction = lstm_model(X_dev)
                        loss = loss_function(prediction, y_dev)
                        
                        # Backpropagation and weights update
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        
                        # Add the loss
                        epoch_loss += loss.item()
                    
                    num_batches = len(train_loader)
                    avg_loss = epoch_loss / num_batches
                    # print(f"Train loss: {avg_loss}")
                
                # Save the model
                model_path = f"lstm_{feature}_{epoch+1}.pth"
                # torch.save(lstm_model.state_dict(), model_path)

                # Calculate out of sample predictions
                outofsample_preds = torch.tensor([]).to(device)
                ground = torch.tensor([]).to(device)
                lstm_model.eval()
                with torch.inference_mode():
                    for X, y in test_loader:
                        X_dev = X.to(device)
                        y_dev = y.to(device)
                        prediction = lstm_model(X_dev)
                        y_unnorm = y_dev * std + mean
                        prediction_unnorm = prediction * std + mean
                        outofsample_preds = torch.cat((outofsample_preds, prediction_unnorm), 0)
                        ground = torch.cat((ground, y_unnorm), 0)

                
                # Calculate insample predictions
                insample_preds = torch.tensor([]).to(device)
                inground = torch.tensor([]).to(device)
                lstm_model.eval()
                with torch.inference_mode():
                    for X, y in test_loader_in_sample:
                        X_dev = X.to(device)
                        y_dev = y.to(device)
                        prediction = lstm_model(X_dev)
                        y_unnorm = y_dev * std + mean
                        prediction_unnorm = prediction * std + mean
                        insample_preds = torch.cat((insample_preds, prediction_unnorm), 0)
                        inground = torch.cat((inground, y_unnorm), 0)
                
                # Write losses in .txt file
                # avg_loss_in = loss_function(inground, insample_preds).item()
                # msg = f'\nMSE loss in sample {feature}: {str(avg_loss_in)}\n'
                # # f.write(msg)
                # print(f'\nMSE loss in sample {feature}: {avg_loss_in}')

                avg_loss_out = loss_function(ground, outofsample_preds).item()
                msg = f'MSE loss out of sample {feature}: {str(avg_loss_out)}\n'
                # f.write(msg)
                print(f'MSE loss out of sample {feature}: {avg_loss_out}\n')
                f.write(f"{i}: {avg_loss_out}\n")

                seq_losses.append(avg_loss_out)

                # Plot insample and out of sample predictions
                
                # plt.figure(figsize=(15, 9))
                # plt.grid(True)
                # plt.plot(inground.cpu(), 'r')
                # plt.plot(insample_preds.cpu(), 'b', marker='o', linestyle='dashed')
                # plt.title(f"LSTM in-sample predictions for {feature}")
                # plt.legend(['training data', 'predictions'])
                # fname= f'lstm_{feature}_insample.png'
                # plt.savefig(fname)
                # plt.show()

                # # to_row = int(len(test_df))
                # sample_range = test_df.index
                # sample_range = torch.tensor(sample_range[:-seq_length-1]).cpu()

                # plt.figure(figsize=(15, 9))
                # plt.grid(True)
                # plt.plot(sample_range, outofsample_preds.cpu(), 'b', marker='o', linestyle='dashed')
                # plt.plot(sample_range, ground.cpu(), 'r')
                # plt.title(f"PREDICTION - {type} - {feature}")
                # plt.xlabel("SAMPLES")
                # plt.ylabel(feature)
                # plt.legend([f'PREDICTED {feature}', f'ACTUAL {feature}'], loc="upper left")
                # fname= f'lstm_{feature}_{type}_predictions.png'
                # plt.savefig(fname)
                # plt.show()

        # plt.figure(figsize=(15, 9))
        # plt.grid(True)
        # plt.plot(range(1,9), seq_losses)
        # plt.title(f"LSTM out-of-sample loss - sequence length for {feature} and type {type}")
        # plt.xlabel("Sequence Length")
        # plt.ylabel("Mean Squared Error")
        # plt.legend(['losses'])
        # fname = f'lstm_{feature}_{type}_losses_seq_lengths.png'
        # plt.savefig(fname)
        # plt.show()




Performing time series analysis for typeA


Training for 50um with sequence length = 1



100%|██████████| 50/50 [00:00<00:00, 292.80it/s]


MSE loss out of sample 50um: 2.6903695389985052e-17


Training for 20um with sequence length = 1



100%|██████████| 50/50 [00:00<00:00, 289.98it/s]


MSE loss out of sample 20um: 3.176448660203246e-17


Performing time series analysis for typeB


Training for 50um with sequence length = 1



...

MSE loss out of sample 20um: 2.163808115897358e-18


Performing time series analysis for typeE


Training for 50um with sequence length = 1



100%|██████████| 50/50 [00:00<00:00, 188.41it/s]


MSE loss out of sample 50um: 2.6346068537234986e-18


Training for 20um with sequence length = 1



100%|██████████| 50/50 [00:00<00:00, 294.71it/s]

MSE loss out of sample 20um: 2.540101055149009e-18

